In [ ]:
#!pip install pyarrow
#!pip install tensorflow

In [ ]:
import helpers
import numpy as np
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset
from torch.optim import lr_scheduler
import pandas as pd
import pyarrow
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
exercise_mapping = {
    'Abduction': 0,
    'Bird': 1,
    'Bridge': 2,
    'Knee': 3,
    'Shoulder': 4,
    'Squat': 5,
    'Stretch': 6
}
set_mapping = {
    'Correct' : 0,
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6
}

In [ ]:
#Importing preprocesssed dataset
train_df = pd.read_parquet('/content/drive/MyDrive/ML_project/RNN/train_set_not_std_less_noise.parquet', engine='pyarrow')
test_df = pd.read_parquet('/content/drive/MyDrive/ML_project/test_set_not_std.parquet', engine='pyarrow')

In [ ]:
video_indices = test_df.groupby('video_id').size().values
video_indices = np.insert(video_indices, 0, 0)
video_indices = np.cumsum(video_indices)
exercise_set_mapping = {exercise_set: index for index, exercise_set in enumerate(train_df[['Exercise', 'Set']].drop_duplicates().itertuples(index=False))}

In [ ]:
get_mlp3x256_ex = lambda: torch.nn.Sequential(

    torch.nn.Flatten(),
    torch.nn.Linear(100, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 7),
)

In [ ]:
get_mlp3x256_set = lambda: torch.nn.Sequential(

    torch.nn.Flatten(),
    torch.nn.Linear(107, 256),
    torch.nn.ReLU(),
    #nn.Dropout(0.1),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    #nn.Dropout(0.1),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    #nn.Dropout(0.1),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 7),
)

In [ ]:
get_custom_mlp = lambda: torch.nn.Sequential(

    nn.Flatten(),
    nn.Linear(107, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Linear(512, 7)
)

In [ ]:
get_mlp3x512_set = lambda: torch.nn.Sequential(

    torch.nn.Flatten(),
    torch.nn.Linear(107, 512),
    torch.nn.ReLU(),
    #nn.Dropout(0.1),
    torch.nn.Linear(512, 512),
    torch.nn.ReLU(),
    #nn.Dropout(0.1),
    torch.nn.Linear(512, 512),
    torch.nn.ReLU(),
    #nn.Dropout(0.1),
    torch.nn.Linear(512, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 7),
)

In [ ]:
get_mlp5x256_set = lambda: torch.nn.Sequential(

    torch.nn.Flatten(),
    torch.nn.Linear(107, 256),
    torch.nn.ReLU(),
    nn.Dropout(0.1),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    nn.Dropout(0.1),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    nn.Dropout(0.1),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    nn.Dropout(0.1),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 7)
)

In [23]:
get_mlp1x512_set = lambda: torch.nn.Sequential(

    torch.nn.Flatten(),
    torch.nn.Linear(107, 512),
    nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    #nn.Dropout(0.1),
    torch.nn.Linear(512, 512),
    nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 7),
)

# Training the model to predict exercises


In [ ]:
X_train, X_test, y_train, y_test = helpers.import_data_exercise(train_df, test_df, exercise_mapping, True)

In [ ]:
learning_rate = 2e-3
# Create an instance of the custom dataset
dataset = TensorDataset(X_train, y_train)

# Create a DataLoader
batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = get_mlp3x256_ex()
model.train()

model_path = '/content/drive/MyDrive/ML_project/MLPEX.path'
#model.load_state_dict(torch.load(model_path))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

In [ ]:
helpers.train_model(1, model, optimizer, criterion, dataloader, scheduler, model_path)
helpers.test_accuracy(model, X_test,y_test,video_indices)

Processing: 100.00%Epoch [1/1], Loss: 0.0611
Accuracy on each frame: 0.9606
Accuracy for videos: 1.0000
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       1.00      1.00      1.00        60
           2       1.00      1.00      1.00        60
           3       1.00      1.00      1.00        60
           4       1.00      1.00      1.00        60
           5       1.00      1.00      1.00        60
           6       1.00      1.00      1.00        60

    accuracy                           1.00       420
   macro avg       1.00      1.00      1.00       420
weighted avg       1.00      1.00      1.00       420



#Training of MLPs to predict sets



In [ ]:
X_train, X_test, y_train, y_test = helpers.import_data_set(train_df, test_df, set_mapping, True)

In [30]:
# Create an instance of the custom dataset
dataset = TensorDataset(X_train, y_train)

# Create a DataLoader
learning_rate = 0.01
batch_size = 256
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = get_mlp1x512_set()
model.train()
model_path = '/content/drive/MyDrive/ML_project/MLP1x512.path'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.3)

In [ ]:
#model_path = '/content/drive/MyDrive/ML_project/CUSMLP5.path'

for i in range(30):
  helpers.train_model(1, model, optimizer, criterion, dataloader, scheduler, model_path)
  helpers.test_accuracy(model, X_test,y_test,video_indices)

Processing: 99.99%Epoch [1/1], Loss: 0.6373
Accuracy on each frame: 0.3927
Accuracy for videos: 0.5548
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.62      0.59       140
           1       0.60      0.54      0.57        76
           2       0.55      0.57      0.56        72
           3       0.61      0.42      0.50        64
           4       0.75      0.53      0.62        40
           5       0.29      0.69      0.41        16
           6       0.38      0.42      0.40        12

    accuracy                           0.55       420
   macro avg       0.54      0.54      0.52       420
weighted avg       0.58      0.55      0.56       420

Processing: 28.87%